In [3]:
import os, glob
import numpy as np
import pandas as pd
import cv2
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import tensorflow as tf
from tensorflow import keras
from keras.optimizers import Adam
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, GlobalAveragePooling2D
from keras import layers
from keras.models import Model
from keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet50_preprocess_input
from keras.applications import EfficientNetB7
from keras.applications.efficientnet import preprocess_input as efficientnet_preprocess_input
from keras.applications import Xception
from keras.applications.xception import preprocess_input as xception_preprocess_input

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# <span style="color: blue;">Read data and save in a dataframe</span>


Colab

In [ ]:
!gdown 1O4YR4UBatOLnaP4gMHbmFw7UJvhhxFwq
!gdown 1-7aMdKW4KcCKLwoUKC3XxdIwfIKkzwx6
! unzip train_data.zip -d /content/train_data
! unzip test_data.zip -d /content/test_data

In [ ]:
train_data_dir = '/content/train_data/train_data'
test_data_dir = '/content/test_data/test_data'
kaggle = False
path=''

Kaggle

In [17]:
train_data_dir = '/kaggle/input/image-prooduct-quera/train_data/train_data'
test_data_dir = '/kaggle/input/image-prooduct-quera/test_data/test_data'
kaggle = True
path='/kaggle/working/'

Save Class Names

In [6]:
class_names = os.listdir(train_data_dir)
class_names

['7', '2', '10', '5', '8', '3', '1', '4', '9', '6']

Read Path of all Train Images

In [7]:
image_dirs = list(glob.glob(train_data_dir + '/**/*.*'))

In [ ]:
len(image_dirs)

Read Label of Train Images

In [8]:
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], image_dirs))

In [ ]:
len(labels)

Create DataFrame of Paths and Labels

In [9]:
data = pd.DataFrame({'image_dir':image_dirs, 'label': labels})
data.head()

,image_dir,label
0,/kaggle/input/image-prooduct-quera/train_data/...,7
1,/kaggle/input/image-prooduct-quera/train_data/...,7
2,/kaggle/input/image-prooduct-quera/train_data/...,7
3,/kaggle/input/image-prooduct-quera/train_data/...,7
4,/kaggle/input/image-prooduct-quera/train_data/...,7


Check Number of Samples in each Class

In [ ]:
counts = data['label'].value_counts()
sns.barplot(x=counts.index, y=counts)
plt.xlabel('Classes')
plt.ylabel('Count')
# plt.xticks(rotation=45)
plt.show()

### Plot some images in each class

In [ ]:
fig, axes = plt.subplots(nrows=5, ncols=4, figsize=(20,25))
for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(data['image_dir'][i * 500]))
    ax.set_title('Class : ' + data['label'][i * 500], fontsize=17)
plt.tight_layout()
plt.show()

# <span style="color: blue;">Define Functions</span>

## Function for generators of train , validation and test

In [21]:
def create_generators(data, 
                      test_size, 
                      preprocessing_function,
                      batch_size = 32,
                      image_size = (224, 224)
                     ):
    
    # Split Data to Tain and Validation
    train, validation = train_test_split(data, test_size=test_size, random_state=42)
    train = train.reset_index(drop=True)
    validation = validation.reset_index(drop=True)
    
    # Create Data Generators
    train_data_generator = ImageDataGenerator(
    preprocessing_function=preprocessing_function,
    rotation_range=15, 
#     width_shift_range=0.2,  
#     height_shift_range=0.2, 
    brightness_range=[0.8,1.2],
    shear_range=0.2,  
    zoom_range=0.2, 
    horizontal_flip=True,
#     vertical_flip=True,
    fill_mode='nearest' 
    )
    
    test_data_generator = ImageDataGenerator(
            preprocessing_function=preprocessing_function
        )

    train_generator = train_data_generator.flow_from_dataframe(
        dataframe= train,
        x_col='image_dir',
        y_col='label',
        target_size=image_size,
        batch_size=batch_size,
        class_mode='categorical', 
        shuffle=True,
        seed=42
    )
    validation_generator = test_data_generator.flow_from_dataframe(
        dataframe= validation,
        x_col='image_dir',
        y_col='label',
        target_size=image_size,
        batch_size=batch_size,
        class_mode='categorical', 
        shuffle=False
    )

    # test_generator = test_data_generator
    
    return train_generator, validation_generator, test_data_generator

In [29]:
def create_generators2(train_data_dir,
                       test_data_dir,
                      preprocessing_function,
                      batch_size = 32,
                      image_size = (224, 224),
                      validation_split=0.3
                     ):
    # Create Data Generators
    train_data_generator = ImageDataGenerator(
        preprocessing_function=preprocessing_function,
        validation_split=validation_split 
    )

    test_data_generator = ImageDataGenerator(
        preprocessing_function=preprocessing_function,
        validation_split=validation_split  
    )

    
    train_generator = train_data_generator.flow_from_directory(
        directory=train_data_dir,
        target_size=image_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True,
        subset='training', 
        seed=42
    )
    
    validation_generator = test_data_generator.flow_from_directory(
        directory=train_data_dir,
        target_size=image_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False,
        subset='validation',  
        seed=42
    )

    return train_generator, validation_generator, test_data_generator

In [26]:
def augment_batch(x_batch, y_batch, data_gen: ImageDataGenerator):
    augmented_images = []
    augmented_labels = []
    
    for x, y in zip(x_batch, y_batch):
        
        augmented_images.append(x)
        augmented_labels.append(y)
        
        x_aug = data_gen.random_transform(x)
        augmented_images.append(x_aug)
        augmented_labels.append(y)
    
    augmented_images = np.array(augmented_images)
    augmented_labels = np.array(augmented_labels)
    
    return augmented_images, augmented_labels

In [27]:
def custom_generator(generator, data_gen):
    for x_batch, y_batch in generator:
        x_batch_aug, y_batch_aug = augment_batch(x_batch, y_batch, data_gen)
        yield x_batch_aug, y_batch_aug

## Functions make pretrained model as base model 

In [11]:
def make_base_model(model=ResNet50,
                    image_size=(224,224),
                    num_classes=10,
                    num_trainable_layeres=0,
                    kaggle=False,
                    weights_path=None,
                   ):
    
    # base_model
    if kaggle:
        # Kaggle
        base_model = model(
        input_shape=(image_size[0], image_size[1], 3),
        classes=num_classes,
        include_top=False, 
        weights=None,
        pooling=None #'avg'
        )
        base_model.load_weights(weights_path)
    else:
        # Colab
        base_model = model(
        input_shape=(image_size[0], image_size[1], 3),
        classes=num_classes,
        include_top=False, 
        weights='imagenet',
        pooling=None #'avg'
        )
    for layer in base_model.layers:
        layer.trainable = False
    if num_trainable_layeres > 0:
        for layer in base_model.layers[-num_trainable_layeres : ]:
            if not isinstance(layer, layers.BatchNormalization):
                layer.trainable = True    
    
    return base_model

## Function for add our layeres to base model

In [32]:
def try_model(base_model, 
              n_added_layers=2, 
              filters=[128,128], 
              activations=['relu', 'relu'], 
              augmentation_layer=False,
              num_classes=10,
              optimizer=Adam(learning_rate=1e-3),
              metrics=['accuracy']):
    if augmentation_layer:
        data_augmentation = keras.Sequential(
        [
        layers.RandomRotation(0.1),
        layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
        layers.RandomFlip(),
        layers.RandomContrast(factor=0.1),
        ]
        )
        inputs = base_model.input
        x = data_augmentation(inputs)
#         scale_layer = keras.layers.Rescaling(scale=1 / 127.5, offset=-1)
#         x = scale_layer(x)
        x = base_model(x, training=False)
        x = GlobalAveragePooling2D()(x)
        x = layers.BatchNormalization()(x)
        x = keras.layers.Dropout(0.2)(x) 
        
    else:
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        x = layers.BatchNormalization()(x)
        x = keras.layers.Dropout(0.2)(x) 
        
    for i in range(n_added_layers):
        x = Dense(filters[i], activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)

    trainable_params = np.sum([K.count_params(p) for p in model.trainable_weights])
    non_trainable_params = np.sum([K.count_params(p) for p in model.non_trainable_weights])
    total_params = trainable_params + non_trainable_params
    
    print(f"Total params: {total_params:,}")
    print(f"Trainable params: {trainable_params:,}")
    print(f"Non-trainable params: {non_trainable_params:,}")
    
    model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=metrics)
#     model.summary()
    return model

## Functions for evaluation metrics 

In [14]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def f1_micro(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    actual_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (actual_positives + K.epsilon())
    
    f1 = 2 * (precision * recall) / (precision + recall + K.epsilon())
    
    return f1

## Functions for save and load models

In [15]:
def save_model(model, filename, path=''):
    filename_json = os.path.join(path, filename + '.json')
    filename_h5 = os.path.join(path, filename + '.h5')

    # serialize model to JSON
    model_json = model.to_json()
    with open(filename_json, "w") as json_file:
        json_file.write(model_json)

    # serialize weights to HDF5
    model.save_weights(filename_h5)
    print("Saved model to ", filename_h5 )
    
def load_model(model, filename, path=''):
    filename_json = os.path.join(path, filename + '.json')
    filename_h5 = os.path.join(path, filename + '.h5')
    
    # load json and create model
    json_file = open(filename_json , 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = keras.models.model_from_json(loaded_model_json)

    # load weights into new model
    loaded_model.load_weights(filename_h5)
    return loaded_model

In [28]:
train_data_generator = ImageDataGenerator(
        rotation_range=20,
        brightness_range=[0.8,1.2],
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

# <span style="color: blue;">Try Models</span>

## <span style="color: green;">ResNet 50</span>

In [ ]:
batch_size = 32
image_size = (224, 224)

train_generator, validation_generator, test_data_generator = create_generators(data, 
                      test_size= 0.3, 
                      preprocessing_function= resnet50_preprocess_input,
                      batch_size = batch_size,
                      image_size = image_size
                     )

### ResNet 50 | train 0 layers 

In [ ]:
resnet50_weights_path = '/kaggle/input/resnet50-weights-notop/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
base_model = make_base_model(model=ResNet50,
                    image_size=image_size,
                    num_classes=len(class_names),
                    num_trainable_layeres=0,
                    kaggle=kaggle,
                    weights_path=resnet50_weights_path,
                   )

model = try_model(base_model, 
              n_added_layers= 0,
              num_classes= len(class_names),
              optimizer= Adam(learning_rate=1e-3),
              metrics= ['accuracy', f1_micro])

early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',
                                               min_delta=0,
                                               patience=5,
                                               restore_best_weights=True)

history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=100,
    callbacks=[early_stopping]
)

# save model
save_model(model, 
           filename='resnet50_train_0_layers', 
           path=path)

### ResNet 50 | train 0 layers | add 2 dense layer with 128 filters

In [ ]:
resnet50_weights_path = '/kaggle/input/resnet50-weights-notop/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
base_model = make_base_model(model=ResNet50,
                    image_size=image_size,
                    num_classes=len(class_names),
                    num_trainable_layeres=0,
                    kaggle=kaggle,
                    weights_path=resnet50_weights_path,
                   )

model = try_model(base_model, 
              n_added_layers= 2, 
              filters= [128,128], 
              activations= ['relu', 'relu'], 
              num_classes= len(class_names),
              optimizer= Adam(learning_rate=1e-3),
              metrics= ['accuracy', f1_micro])

early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',
                                               min_delta=0,
                                               patience=5,
                                               restore_best_weights=True)

history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=100,
    callbacks=[early_stopping]
)

# save model
save_model(model, 
           filename='resnet50_train_0_add_2_layers', 
           path=path)

### ResNet 50 | train 5 last layers | add 2 dense layers both with 128 filters

In [ ]:
resnet50_weights_path = '/kaggle/input/resnet50-weights-notop/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
base_model = make_base_model(model=ResNet50,
                    image_size=image_size,
                    num_classes=len(class_names),
                    num_trainable_layeres=5,
                    kaggle=kaggle,
                    weights_path=resnet50_weights_path,
                   )

model = try_model(base_model, 
              n_added_layers= 2, 
              filters= [128,128], 
              activations= ['relu', 'relu'], 
              num_classes= len(class_names),
              optimizer= Adam(learning_rate=1e-3),
              metrics= ['accuracy', f1_micro])

early_stopping = keras.callbacks.EarlyStopping(monitor='val_f1_micro',
                                               min_delta=0,
                                               patience=5,
                                               mode='max',
                                               restore_best_weights=True)

history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=100,
    callbacks=[early_stopping]
)

# save model
save_model(model, 
           filename='resnet50_train_5_add_2_layers_128_128', 
           path=path)

### ResNet 50 | train 10 last layers | add 2 dense layer with 256 and 128 filters

In [ ]:
resnet50_weights_path = '/kaggle/input/resnet50-weights-notop/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
base_model = make_base_model(model=ResNet50,
                    image_size=image_size,
                    num_classes=len(class_names),
                    num_trainable_layeres=10,
                    kaggle=kaggle,
                    weights_path=resnet50_weights_path,
                   )

model = try_model(base_model, 
              n_added_layers= 2, 
              filters= [256,128], 
              activations= ['relu', 'relu'], 
              num_classes= len(class_names),
              optimizer= Adam(learning_rate=1e-3),
              metrics= ['accuracy', f1_micro])

early_stopping = keras.callbacks.EarlyStopping(monitor='val_f1_micro',
                                               min_delta=0,
                                               patience=5,
                                               mode='max',
                                               restore_best_weights=True)

history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=100,
    callbacks=[early_stopping]
)

# save model
save_model(model, 
           filename='resnet50_train_10_add_2_layers_256_128', 
           path=path)

### ResNet 50 | train 15 last layers | add 2 dense layer with 256 and 128 filters

In [ ]:
resnet50_weights_path = '/kaggle/input/resnet50-weights-notop/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
base_model = make_base_model(model=ResNet50,
                    image_size=image_size,
                    num_classes=len(class_names),
                    num_trainable_layeres=15,
                    kaggle=kaggle,
                    weights_path=resnet50_weights_path,
                   )

model = try_model(base_model, 
              n_added_layers= 2, 
              filters= [256,128], 
              activations= ['relu', 'relu'], 
              num_classes= len(class_names),
              optimizer= Adam(learning_rate=1e-3),
              metrics= ['accuracy', f1_micro])

early_stopping = keras.callbacks.EarlyStopping(monitor='val_f1_micro',
                                               min_delta=0,
                                               patience=5,
                                               mode='max',
                                               restore_best_weights=True)

history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=100,
    callbacks=[early_stopping]
)

# save model
save_model(model, 
           filename='resnet50_train_15_add_2_layers_256_128', 
           path=path)

### ResNet 50 | train 20 last layers | add 2 dense layer with 256 and 128 filters | lr : 1e-4

In [ ]:
resnet50_weights_path = '/kaggle/input/resnet50-weights-notop/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
base_model = make_base_model(model=ResNet50,
                    image_size=image_size,
                    num_classes=len(class_names),
                    num_trainable_layeres=20,
                    kaggle=kaggle,
                    weights_path=resnet50_weights_path,
                   )

model = try_model(base_model, 
              n_added_layers= 2, 
              filters= [256,128], 
              activations= ['relu', 'relu'], 
              num_classes= len(class_names),
              optimizer= Adam(learning_rate=1e-4),
              metrics= ['accuracy', f1_micro])

early_stopping = keras.callbacks.EarlyStopping(monitor='val_f1_micro',
                                               min_delta=0,
                                               patience=5,
                                               mode='max',
                                               restore_best_weights=True)

history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=100,
    callbacks=[early_stopping]
)

# save model
save_model(model, 
           filename='resnet50_train_20_add_2_layers_256_128_lr-4', 
           path=path)

### ResNet 50 | train 25 last layers | add 2 dense layer with 256 and 128 filters | lr : 1e-4

In [ ]:
resnet50_weights_path = '/kaggle/input/resnet50-weights-notop/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
base_model = make_base_model(model=ResNet50,
                    image_size=image_size,
                    num_classes=len(class_names),
                    num_trainable_layeres=25,
                    kaggle=kaggle,
                    weights_path=resnet50_weights_path,
                   )

model = try_model(base_model, 
              n_added_layers= 2, 
              filters= [256,128], 
              activations= ['relu', 'relu'], 
              num_classes= len(class_names),
              optimizer= Adam(learning_rate=1e-4),
              metrics= ['accuracy', f1_micro])

early_stopping = keras.callbacks.EarlyStopping(monitor='val_f1_micro',
                                               min_delta=0,
                                               patience=5,
                                               mode='max',
                                               restore_best_weights=True)

history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=100,
    callbacks=[early_stopping]
)

# save model
save_model(model, 
           filename='resnet50_train_25_add_2_layers_256_128_lr-4', 
           path=path)

In [ ]:
resnet50_weights_path = '/kaggle/input/resnet50-weights-notop/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
base_model = make_base_model(model=ResNet50,
                    image_size=image_size,
                    num_classes=len(class_names),
                    num_trainable_layeres=45,
                    kaggle=kaggle,
                    weights_path=resnet50_weights_path,
                   )

model = try_model(base_model, 
              n_added_layers= 2, 
              filters= [256,128], 
              activations= ['relu', 'relu'], 
              num_classes= len(class_names),
              optimizer= Adam(learning_rate=1e-4),
              metrics= ['accuracy', f1_micro])

early_stopping = keras.callbacks.EarlyStopping(monitor='val_f1_micro',
                                               min_delta=0,
                                               patience=5,
                                               mode='max',
                                               restore_best_weights=True)

history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=100,
    callbacks=[early_stopping]
)

# save model
save_model(model, 
           filename='resnet50_train_45_add_2_layers_256_128_lr-4', 
           path=path)

In [ ]:
resnet50_weights_path = '/kaggle/input/resnet50-weights-notop/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
base_model = make_base_model(model=ResNet50,
                    image_size=image_size,
                    num_classes=len(class_names),
                    num_trainable_layeres=20,
                    kaggle=kaggle,
                    weights_path=resnet50_weights_path,
                   )

model = try_model(base_model, 
              n_added_layers= 2, 
              filters= [256,128], 
              activations= ['relu', 'relu'], 
              num_classes= len(class_names),
              optimizer= Adam(learning_rate=1e-4),
              metrics= ['accuracy', f1_micro])

early_stopping = keras.callbacks.EarlyStopping(monitor='val_f1_micro',
                                               min_delta=0,
                                               patience=5,
                                               mode='max',
                                               restore_best_weights=True)

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=100,
    callbacks=[early_stopping]
)

# save model
save_model(model, 
           filename='resnet50_train_20_add_2_layers_256_128_lr-4', 
           path=path)

## <span style="color: green;">EfficientNet</span>

In [ ]:
batch_size = 32
image_size = (224, 224)
train_generator, validation_generator, test_data_generator = create_generators(data, 
                      test_size= 0.3, 
                      preprocessing_function= efficientnet_preprocess_input,
                      batch_size = batch_size,
                      image_size = image_size
                     )

In [ ]:
efficientnet_weights_path = '/kaggle/input/efficientnetb7-weights-notop/efficientnetb7_notop.h5'
base_model = make_base_model(model=EfficientNetB7,
                    image_size=image_size,
                    num_classes=len(class_names),
                    num_trainable_layeres=20,
                    kaggle=kaggle,
                    weights_path=efficientnet_weights_path,
                   )

model = try_model(base_model, 
              n_added_layers= 2, 
              filters= [256,128], 
              activations= ['relu', 'relu'], 
              num_classes= len(class_names),
              optimizer= Adam(learning_rate=1e-4),
              metrics= ['accuracy', f1_micro])

early_stopping = keras.callbacks.EarlyStopping(monitor='val_f1_micro',
                                               min_delta=0,
                                               patience=5,
                                               mode='max',
                                               restore_best_weights=True)

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=100,
    callbacks=[early_stopping]
)

# save model
save_model(model, 
           filename='efficientnet_train_20_add_2_layers_256_128_lr-4', 
           path=path)

## <span style="color: green;">Xception</span>

In [22]:
batch_size = 32
image_size = (299, 299)

train_generator, validation_generator, test_data_generator = create_generators(data, 
                      test_size= 0.3, 
                      preprocessing_function= xception_preprocess_input,
                      batch_size = batch_size,
                      image_size = image_size
                     )

Found 6993 validated image filenames belonging to 10 classes.
Found 2997 validated image filenames belonging to 10 classes.


In [30]:
train_generator, validation_generator, test_data_generator = create_generators2(train_data_dir,
                      test_data_dir,
                      validation_split=0.3,
                      preprocessing_function= xception_preprocess_input,
                      batch_size = batch_size,
                      image_size = image_size
                      
                     )

Found 6994 images belonging to 10 classes.
Found 2996 images belonging to 10 classes.


In [ ]:
xception_weights_path = '/kaggle/input/xception-weights-notop/xception-weights-notop.h5'
base_model = make_base_model(model=Xception,
                    image_size=image_size,
                    num_classes=len(class_names),
                    num_trainable_layeres=20,
                    kaggle=kaggle,
                    weights_path=xception_weights_path,
                   )

model = try_model(base_model, 
              n_added_layers= 2, 
              filters= [256,128], 
              activations= ['relu', 'relu'], 
              num_classes= len(class_names),
              optimizer= Adam(learning_rate=1e-4),
              metrics= ['accuracy', f1_micro])

early_stopping = keras.callbacks.EarlyStopping(monitor='val_f1_micro',
                                               min_delta=0,
                                               patience=5,
                                               mode='max',
                                               restore_best_weights=True)

history = model.fit(
    custom_generator(train_generator, train_data_generator),
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=50,
    callbacks=[early_stopping]
)

# save model
save_model(model, 
           filename='xception_train_20_add_2_layers_256_128_lr-4', 
           path=path)

Total params: 21,428,402
Trainable params: 7,875,026
Non-trainable params: 13,553,376
Epoch 1/50
 93/218 [===========>..................] - ETA: 2:04 - loss: 1.8111 - accuracy: 0.3967 - f1_micro: 0.2819

In [ ]:
xception_weights_path = '/kaggle/input/xception-weights-notop/xception-weights-notop.h5'
base_model = make_base_model(model=Xception,
                    image_size=image_size,
                    num_classes=len(class_names),
                    num_trainable_layeres=20,
                    kaggle=kaggle,
                    weights_path=xception_weights_path,
                   )

model = try_model(base_model, 
              n_added_layers= 2, 
              filters= [256,128], 
              activations= ['relu', 'relu'], 
              num_classes= len(class_names),
              optimizer= Adam(learning_rate=1e-4),
              metrics= ['accuracy', f1_micro])

early_stopping = keras.callbacks.EarlyStopping(monitor='val_f1_micro',
                                               min_delta=0,
                                               patience=5,
                                               mode='max',
                                               restore_best_weights=True)

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=50,
    callbacks=[early_stopping]
)

# save model
save_model(model, 
           filename='xception_train_20_add_2_layers_256_128_lr-4', 
           path=path)